In [3]:
# Install PyTorch and other packages (in Colab environment)
!pip install torch torchvision albumentations opencv-python tqdm

import os
import math
import torch
import random
import numpy as np
import cv2
from glob import glob
from PIL import Image
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from tqdm import tqdm

# Device configuration (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda


In [2]:
# ============================================================
# Install and download dataset via KaggleHub
# ============================================================
!pip install -q kagglehub

import kagglehub, os, shutil, numpy as np
from PIL import Image

print("Downloading WIDER FACE dataset from Kaggle...")
kaggle_path = kagglehub.dataset_download("iamprateek/wider-face-a-face-detection-dataset")
print(f" Original KaggleHub path: {kaggle_path}")

# ============================================================
# Move dataset into /content for easy access
# ============================================================
target_dir = "/content/WIDERFace"
if os.path.exists(target_dir):
    shutil.rmtree(target_dir)
shutil.copytree(kaggle_path, target_dir)
print(f" Dataset copied to: {target_dir}")

# ============================================================
# Auto-detect nested folder structure
# ============================================================
# Detect actual image and annotation locations
train_dir = None
val_dir = None
ann_dir = None

for root, dirs, files in os.walk(target_dir):
    if "WIDER_train" in root and "images" in dirs:
        train_dir = os.path.join(root, "images")
    elif "WIDER_val" in root and "images" in dirs:
        val_dir = os.path.join(root, "images")
    elif "wider_face_annotations" in root:
        ann_dir = root

print("\n Detected paths:")
print("Train Images:", train_dir)
print("Val Images:  ", val_dir)
print("Annotations:", ann_dir)

# ============================================================
#  Safe annotation loader
# ============================================================
def load_wider_annotations(split='train', ann_root=ann_dir):
    """Load WIDER FACE annotations safely from Kaggle dataset."""
    ann_file = os.path.join(ann_root, f"wider_face_{split}_bbx_gt.txt")
    if not os.path.exists(ann_file):
        print(f" Annotation file not found: {ann_file}")
        return []

    lines = open(ann_file).read().strip().splitlines()
    idx = 0
    annotations = []
    while idx < len(lines):
        img_path = lines[idx].strip()
        idx += 1
        try:
            num_boxes = int(lines[idx].strip())
            idx += 1
        except ValueError:
            while idx < len(lines) and not lines[idx].strip().endswith('.jpg'):
                idx += 1
            continue

        boxes = []
        for _ in range(num_boxes):
            if idx >= len(lines): break
            vals = list(map(int, lines[idx].split()))
            if len(vals) >= 4 and vals[2] > 0 and vals[3] > 0:
                boxes.append([vals[0], vals[1], vals[2], vals[3]])
            idx += 1
        annotations.append((img_path, np.array(boxes, dtype=np.float32)))
    return annotations

# ============================================================
# Load annotations and verify
# ============================================================
train_annotations = load_wider_annotations('train', ann_root=ann_dir)
val_annotations   = load_wider_annotations('val', ann_root=ann_dir)

print(f"\n Loaded {len(train_annotations)} training images")
print(f" Loaded {len(val_annotations)} validation images")

# ============================================================
# Test a random image
# ============================================================
# The dataset stores paths like: 0--Parade/0_Parade_marchingband_1_1038.jpg
# so we just prepend our detected train_dir to access the actual file.
if len(train_annotations) > 0:
    sample_img, _ = train_annotations[100]
    full_path = os.path.join(train_dir, sample_img)
    print(f"Example image path: {full_path}")
    Image.open(full_path).show()
else:
    print("No training annotations loaded. Check annotation directory structure.")


100%|██████████| 3.43G/3.43G [00:38<00:00, 94.7MB/s]

Extracting files...


 Original KaggleHub path: /root/.cache/kagglehub/datasets/iamprateek/wider-face-a-face-detection-dataset/versions/1
 Dataset copied to: /content/WIDERFace

 Detected paths:
Train Images: /content/WIDERFace/WIDER_train/WIDER_train/images
Val Images:   /content/WIDERFace/WIDER_val/WIDER_val/images
Annotations: /content/WIDERFace/wider_face_annotations/wider_face_split

 Loaded 12880 training images
 Loaded 3226 validation images
Example image path: /content/WIDERFace/WIDER_train/WIDER_train/images/0--Parade/0_Parade_marchingband_1_1038.jpg


In [4]:
# Define transformations (resize and conversion to tensor)
input_size = 300
to_tensor = transforms.ToTensor()

def preprocess(image, boxes):
    """
    Safely resize and normalize image, scaling boxes to input size.
    Handles empty or malformed annotations robustly.
    """
    orig_w, orig_h = image.size
    image = image.resize((input_size, input_size))
    scale_x = input_size / orig_w
    scale_y = input_size / orig_h

    # ---- Fix starts here ----
    # Normalize malformed box arrays
    if boxes is None or len(boxes) == 0:
        boxes = np.zeros((0, 4), dtype=np.float32)
    boxes = np.array(boxes, dtype=np.float32)

    # If 1D (like [x,y,w,h]) reshape into (1,4)
    if boxes.ndim == 1:
        if boxes.size == 4:
            boxes = boxes.reshape(1, 4)
        else:
            boxes = np.zeros((0, 4), dtype=np.float32)

    # Filter invalid or zero-sized boxes
    valid = (boxes[:, 2] > 0) & (boxes[:, 3] > 0)
    boxes = boxes[valid]
    if boxes.size == 0:
        boxes = np.zeros((0, 4), dtype=np.float32)
    # ---- Fix ends here ----

    # Convert (x, y, w, h) → (xmin, ymin, xmax, ymax)
    boxes[:, 2] = boxes[:, 2] + boxes[:, 0]
    boxes[:, 3] = boxes[:, 3] + boxes[:, 1]

    # Scale coordinates
    boxes[:, 0] *= scale_x
    boxes[:, 1] *= scale_y
    boxes[:, 2] *= scale_x
    boxes[:, 3] *= scale_y

    # Random horizontal flip (data augmentation)
    if random.random() < 0.5 and len(boxes) > 0:
        image = transforms.functional.hflip(image)
        boxes[:, [0, 2]] = input_size - boxes[:, [2, 0]]

    # Convert to tensor format
    img_tensor = to_tensor(image)
    target = {
        'boxes': torch.tensor(boxes, dtype=torch.float32),
        'labels': torch.ones((boxes.shape[0],), dtype=torch.int64)
    }
    return img_tensor, target



In [5]:
class WIDERFaceDataset(Dataset):
    def __init__(self, annotations, images_root, transform=None):
        self.annotations = annotations
        self.images_root = images_root
        self.transform = transform
    def __len__(self):
        return len(self.annotations)
    def __getitem__(self, idx):
        img_path, boxes = self.annotations[idx]
        img = Image.open(os.path.join(self.images_root, img_path)).convert("RGB")
        if self.transform:
            img_tensor, target = self.transform(img, boxes)
        else:
            img_tensor, target = to_tensor(img), {'boxes': torch.tensor(boxes, dtype=torch.float32),
                                                  'labels': torch.ones((len(boxes),), dtype=torch.int64)}
        return img_tensor, target

# Create dataset and dataloader
train_dataset = WIDERFaceDataset(train_annotations, images_root="/content/WIDERFace/WIDER_train/WIDER_train/images", transform=preprocess)
val_dataset   = WIDERFaceDataset(val_annotations,   images_root="/content/WIDERFace/WIDER_val/WIDER_val/images",   transform=preprocess)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=lambda b: tuple(zip(*b)))
val_loader   = DataLoader(val_dataset,   batch_size=8, shuffle=False, collate_fn=lambda b: tuple(zip(*b)))


In [6]:
class FaceDetectorSSD(nn.Module):
    def __init__(self, num_anchors=4):
        super().__init__()
        # Define a small backbone with depthwise separable convs
        def conv_bn_relu(in_ch, out_ch, stride=1):
            return nn.Sequential(
                nn.Conv2d(in_ch, out_ch, 3, stride=stride, padding=1, bias=False),
                nn.BatchNorm2d(out_ch), nn.ReLU(inplace=True)
            )
        # Depthwise separable conv block
        class DWBlock(nn.Module):
            def __init__(self, in_ch, out_ch, stride=1):
                super().__init__()
                self.dw = nn.Conv2d(in_ch, in_ch, 3, stride=stride, padding=1, groups=in_ch, bias=False)
                self.bn1 = nn.BatchNorm2d(in_ch)
                self.pw = nn.Conv2d(in_ch, out_ch, 1, bias=False)
                self.bn2 = nn.BatchNorm2d(out_ch)
                self.act = nn.ReLU(inplace=True)
            def forward(self, x):
                x = self.act(self.bn1(self.dw(x)))
                x = self.act(self.bn2(self.pw(x)))
                return x

        # Backbone layers
        self.layer1 = conv_bn_relu(3,  32, stride=2)  # 300x300 -> 150x150
        self.layer2 = DWBlock(32, 32)                  # 150x150
        self.layer3 = conv_bn_relu(32, 64, stride=2)  # 150x150 -> 75x75
        self.layer4 = DWBlock(64, 64)                  # 75x75
        self.layer5 = conv_bn_relu(64,128, stride=2)  # 75x75 -> 38x38
        self.layer6 = DWBlock(128,128)                 # 38x38 final feature map

        # Detection head: classification and regression
        self.cls_head = nn.Conv2d(128, num_anchors,   3, padding=1)  # one score per anchor
        self.reg_head = nn.Conv2d(128, num_anchors*4, 3, padding=1)  # 4 coords per anchor

    def forward(self, x):
        # Backbone forward
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = self.layer6(x)  # final feature map (N,128,H,W)

        # Predict scores and box offsets
        cls_out = self.cls_head(x)  # (N, num_anchors, H, W)
        reg_out = self.reg_head(x)  # (N, num_anchors*4, H, W)

        # Rearrange to shape (N, H*W*num_anchors, 1) and (N, H*W*num_anchors, 4)
        N, A, H, W = cls_out.shape
        cls_out = cls_out.permute(0,2,3,1).contiguous().view(N, -1)  # flatten scores
        reg_out = reg_out.view(N, A, 4, H, W).permute(0,3,4,1,2).contiguous().view(N, -1, 4)
        return cls_out, reg_out

# Instantiate the model
num_anchors = 4  # e.g. 4 default boxes per cell
model = FaceDetectorSSD(num_anchors=num_anchors).to(device)


In [7]:
# ============================================================
#  Face Detector (Multi-Ratio Anchors, Focal Loss, Checkpointing)
# ============================================================

import torch, torch.nn as nn, torch.nn.functional as F
import math, os
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_size = 300

# ------------------------------------------------------------
# Model Definition
# ------------------------------------------------------------
class FaceDetector(nn.Module):
    def __init__(self, num_anchor_scales=5, num_anchor_ratios=3):
        super().__init__()
        base_channels = 64
        self.backbone = nn.Sequential(
            nn.Conv2d(3, base_channels, 3, stride=2, padding=1),  # 150x150
            nn.ReLU(inplace=True),
            nn.Conv2d(base_channels, base_channels * 2, 3, stride=2, padding=1),  # 75x75
            nn.ReLU(inplace=True),
            nn.Conv2d(base_channels * 2, base_channels * 4, 3, stride=2, padding=1),  # 38x38
            nn.ReLU(inplace=True),
        )

        self.num_anchors = num_anchor_scales * num_anchor_ratios  # 5x3 = 15 anchors per cell

        # prediction heads
        self.cls_head = nn.Conv2d(base_channels * 4, self.num_anchors * 1, 3, padding=1)
        self.reg_head = nn.Conv2d(base_channels * 4, self.num_anchors * 4, 3, padding=1)

        for m in [self.cls_head, self.reg_head]:
            nn.init.xavier_uniform_(m.weight)
            nn.init.constant_(m.bias, 0.)

    def forward(self, x):
        x = self.backbone(x)                      # (B,256,38,38)
        B = x.size(0)
        A = self.num_anchors

        cls_out = self.cls_head(x)                # (B,A,38,38)
        reg_out = self.reg_head(x)                # (B,Ax4,38,38)

        cls_out = cls_out.permute(0,2,3,1).contiguous().view(B, -1)
        reg_out = reg_out.permute(0,2,3,1).contiguous().view(B, -1, 4)
        return cls_out, reg_out


# ------------------------------------------------------------
# Anchor Generator
# ------------------------------------------------------------
feature_H, feature_W = 38, 38
stride = input_size / feature_H
anchor_sizes = [16, 32, 64, 128, 256]
ratios = [0.5, 1.0, 2.0]

def make_anchors(feature_size, stride, sizes, ratios):
    H, W = feature_size
    anchors = []
    for i in range(H):
        for j in range(W):
            cx, cy = (j + 0.5) * stride, (i + 0.5) * stride
            for s in sizes:
                for r in ratios:
                    w = s * math.sqrt(r)
                    h = s / math.sqrt(r)
                    anchors.append([cx, cy, w, h])
    return torch.tensor(anchors, dtype=torch.float32)

anchors = make_anchors((feature_H, feature_W), stride, anchor_sizes, ratios).to(device)
num_total_anchors = anchors.size(0)
print(f"✅ Generated {num_total_anchors} anchors ({len(anchor_sizes)} scales × {len(ratios)} ratios)")

# ------------------------------------------------------------
# IoU Function
# ------------------------------------------------------------
def box_iou(boxes1, boxes2):
    area1 = (boxes1[:, 2] - boxes1[:, 0]) * (boxes1[:, 3] - boxes1[:, 1])
    area2 = (boxes2[:, 2] - boxes2[:, 0]) * (boxes2[:, 3] - boxes2[:, 1])
    lt = torch.max(boxes1[:, None, :2], boxes2[:, :2])
    rb = torch.min(boxes1[:, None, 2:], boxes2[:, 2:])
    wh = (rb - lt).clamp(min=0)
    inter = wh[:,:,0] * wh[:,:,1]
    iou = inter / (area1[:,None] + area2 - inter + 1e-8)
    return iou

# ------------------------------------------------------------
# Focal Loss
# ------------------------------------------------------------
def focal_loss(pred, target, alpha=0.25, gamma=2.0):
    prob = torch.sigmoid(pred)
    ce = F.binary_cross_entropy_with_logits(pred, target, reduction='none')
    p_t = prob * target + (1 - prob) * (1 - target)
    loss = alpha * (1 - p_t) ** gamma * ce
    return loss

# ------------------------------------------------------------
# Initialize Model & Optimizer
# ------------------------------------------------------------
model = FaceDetector(num_anchor_scales=len(anchor_sizes), num_anchor_ratios=len(ratios)).to(device)
print(f"✅ Model initialized: {model.num_anchors} anchors/cell × {feature_H}×{feature_W} = {model.num_anchors*feature_H*feature_W} total")

optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=5e-4)
reg_loss_fn = nn.SmoothL1Loss(reduction='none')

# ------------------------------------------------------------
# Checkpoint Setup
# ------------------------------------------------------------
save_dir = "/content/models"
os.makedirs(save_dir, exist_ok=True)
checkpoint_path = os.path.join(save_dir, "latest_face_detector_multi_ratio.pth")

start_epoch = 0
best_loss = float("inf")
if os.path.exists(checkpoint_path):
    checkpoint = torch.load(checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint["model_state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
    start_epoch = checkpoint["epoch"]
    best_loss = checkpoint.get("best_loss", best_loss)
    print(f"🔁 Resumed training from epoch {start_epoch} (best_loss={best_loss:.4f})")

# ------------------------------------------------------------
# Training Loop
# ------------------------------------------------------------
num_epochs = 50

for epoch in range(start_epoch, num_epochs):
    model.train()
    total_loss = 0.0

    for imgs, targets in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        imgs = torch.stack(imgs).to(device)
        batch_size = imgs.size(0)
        cls_preds, reg_preds = model(imgs)

        all_cls_loss, all_reg_loss, total_pos = 0.0, 0.0, 0

        for b in range(batch_size):
            gt_boxes = targets[b]['boxes'].to(device)
            if gt_boxes.numel() == 0:
                continue

            anchor_boxes_xyxy = torch.cat([
                anchors[:, :2] - anchors[:, 2:] / 2,
                anchors[:, :2] + anchors[:, 2:] / 2
            ], dim=1)
            ious = box_iou(anchor_boxes_xyxy, gt_boxes)

            best_anchor_idx = ious.argmax(dim=0)
            max_iou_anchor, _ = ious.max(dim=1)

            labels = torch.full((num_total_anchors,), -1.0, dtype=torch.float32, device=device)
            labels[max_iou_anchor < 0.4] = 0.0
            labels[best_anchor_idx] = 1.0
            labels[max_iou_anchor >= 0.5] = 1.0

            cls_pred = cls_preds[b].view(-1)
            mask = (labels != -1)
            if mask.sum() > 0:
                cls_loss = focal_loss(cls_pred[mask], labels[mask]).sum()
                all_cls_loss += cls_loss

            pos_idx = torch.where(labels == 1)[0]
            total_pos += pos_idx.numel()
            if pos_idx.numel() > 0:
                anchors_pos = anchors[pos_idx]
                assigned_gt = gt_boxes[ious[pos_idx].argmax(dim=1)]
                anc_cx, anc_cy, anc_w, anc_h = anchors_pos.t()
                gt_cx = 0.5*(assigned_gt[:,0]+assigned_gt[:,2])
                gt_cy = 0.5*(assigned_gt[:,1]+assigned_gt[:,3])
                gt_w  = assigned_gt[:,2]-assigned_gt[:,0]
                gt_h  = assigned_gt[:,3]-assigned_gt[:,1]
                tx = (gt_cx - anc_cx) / anc_w
                ty = (gt_cy - anc_cy) / anc_h
                tw = torch.log(gt_w / anc_w + 1e-6)
                th = torch.log(gt_h / anc_h + 1e-6)
                reg_targets = torch.stack([tx, ty, tw, th], dim=1)
                reg_pred = reg_preds[b][pos_idx]
                reg_loss = reg_loss_fn(reg_pred, reg_targets).sum()
                all_reg_loss += reg_loss

        loss = (all_cls_loss + all_reg_loss) / max(total_pos, 1)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_loss = total_loss / max(len(train_loader), 1)
    print(f"📘 Epoch {epoch+1}/{num_epochs} | Avg Loss: {avg_loss:.4f}")

    # 💾 Save Checkpoints
    torch.save({
        "epoch": epoch + 1,
        "model_state_dict": model.state_dict(),
        "optimizer_state_dict": optimizer.state_dict(),
        "best_loss": best_loss,
    }, checkpoint_path)
    print(f"✅ Saved checkpoint → {checkpoint_path}")

    if avg_loss < best_loss:
        best_loss = avg_loss
        best_path = os.path.join(save_dir, "best_face_detector_multi_ratio.pth")
        torch.save(model.state_dict(), best_path)
        print(f"🌟 New best model saved (Loss={best_loss:.4f})")

print("🎯 Training completed successfully!")


✅ Generated 21660 anchors (5 scales × 3 ratios)
✅ Model initialized: 15 anchors/cell × 38×38 = 21660 total


Epoch 1/50: 100%|██████████| 1610/1610 [05:23<00:00,  4.98it/s]


📘 Epoch 1/50 | Avg Loss: 0.9138
✅ Saved checkpoint → /content/models/latest_face_detector_multi_ratio.pth
🌟 New best model saved (Loss=0.9138)


Epoch 2/50: 100%|██████████| 1610/1610 [05:02<00:00,  5.32it/s]


📘 Epoch 2/50 | Avg Loss: 0.6796
✅ Saved checkpoint → /content/models/latest_face_detector_multi_ratio.pth
🌟 New best model saved (Loss=0.6796)


...

📘 Epoch 48/50 | Avg Loss: 0.4512
✅ Saved checkpoint → /content/models/latest_face_detector_multi_ratio.pth


Epoch 49/50: 100%|██████████| 1610/1610 [04:55<00:00,  5.45it/s]


📘 Epoch 49/50 | Avg Loss: 0.4500
✅ Saved checkpoint → /content/models/latest_face_detector_multi_ratio.pth


Epoch 50/50: 100%|██████████| 1610/1610 [04:59<00:00,  5.37it/s]

📘 Epoch 50/50 | Avg Loss: 0.4485
✅ Saved checkpoint → /content/models/latest_face_detector_multi_ratio.pth
🎯 Training completed successfully!


In [8]:
# Load model checkpoint
# checkpoint = torch.load("/content/models/face_detector_epoch_10.pth", map_location=device)

# model.load_state_dict(checkpoint['model_state_dict'])
# optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
# start_epoch = checkpoint['epoch']
# print(f" Loaded model from epoch {start_epoch}")


In [9]:
model.eval()
with torch.no_grad():
    all_tp = all_fp = all_fn = 0
    for imgs, targets in val_loader:
        imgs = torch.stack(imgs).to(device)
        cls_preds, reg_preds = model(imgs)
        batch_size = imgs.size(0)
        for b in range(batch_size):
            # Ground truth boxes for this image
            gt_boxes = targets[b]['boxes'].numpy()
            # Decode predictions: apply sigmoid to scores
            scores = torch.sigmoid(cls_preds[b]).cpu().numpy()
            boxes = anchors.cpu().numpy()  # anchors [cx,cy,w,h]
            # Decode predicted offsets
            deltas = reg_preds[b].cpu().numpy()  # (A_total,4)
            # Decode boxes: [cx,cy,w,h] + offsets -> [xmin,ymin,xmax,ymax]
            cx = boxes[:,0] + deltas[:,0]*boxes[:,2]
            cy = boxes[:,1] + deltas[:,1]*boxes[:,3]
            w  = boxes[:,2] * np.exp(deltas[:,2])
            h  = boxes[:,3] * np.exp(deltas[:,3])
            # Convert to corners
            pred_boxes = np.stack([cx - w/2, cy - h/2, cx + w/2, cy + h/2], axis=1)
            # Filter by score threshold
            threshold = 0.5
            mask = scores > threshold
            pred_boxes = pred_boxes[mask]
            pred_scores = scores[mask]

            # For simplicity, apply no NMS (assume faces are sparse).
            # Otherwise run a greedy NMS to remove duplicates.

            # Match predicted boxes to GT
            if len(pred_boxes) == 0:
                all_fn += gt_boxes.shape[0]
                continue
            if gt_boxes.shape[0] == 0:
                all_fp += len(pred_boxes)
                continue
            # Compute IoU matrix
            ious = box_iou(
                torch.tensor(pred_boxes, dtype=torch.float32),
                torch.tensor(gt_boxes, dtype=torch.float32)
            ).numpy()
            # Determine matches
            assigned_gt = -np.ones(len(pred_boxes), dtype=int)
            for i in range(len(pred_boxes)):
                j = np.argmax(ious[i])
                if ious[i,j] >= 0.5:
                    assigned_gt[i] = j
            # Count true positives (one per GT)
            matched_gt = set()
            tp = fp = fn = 0
            for i, gt_idx in enumerate(assigned_gt):
                if gt_idx >= 0 and gt_idx not in matched_gt:
                    tp += 1
                    matched_gt.add(gt_idx)
                else:
                    fp += 1
            fn = gt_boxes.shape[0] - len(matched_gt)
            all_tp += tp
            all_fp += fp
            all_fn += fn

    precision = all_tp / (all_tp + all_fp + 1e-8)
    recall    = all_tp / (all_tp + all_fn + 1e-8)
    print(f"Validation Precision: {precision:.3f}, Recall: {recall:.3f}")


Validation Precision: 0.461, Recall: 0.027


In [ ]:
import time
while True:
    time.sleep(60)  # Waits for 60 seconds


In [ ]:
# ============================================================
# 🎨 Inference & Visualization for Face Detector
# ============================================================

import torch
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ------------------------------------------------------------
# Load Model + Anchors
# ------------------------------------------------------------
best_model_path = "/content/models/best_face_detector_multi_ratio.pth"

model = FaceDetector(num_anchor_scales=len(anchor_sizes), num_anchor_ratios=len(ratios)).to(device)
model.load_state_dict(torch.load(best_model_path, map_location=device))
model.eval()
print(f"Loaded model from: {best_model_path}")

# ------------------------------------------------------------
# Helper: Decode Anchors + Predictions
# ------------------------------------------------------------
def decode_predictions(cls_pred, reg_pred, anchors, conf_thresh=0.5):
    """
    Decode model outputs into bounding boxes and confidences.
    """
    probs = torch.sigmoid(cls_pred)
    mask = probs > conf_thresh
    if mask.sum() == 0:
        return []

    probs = probs[mask]
    boxes = anchors[mask]
    reg = reg_pred[mask]

    # Decode center/size offsets back to (xmin, ymin, xmax, ymax)
    cx = boxes[:, 0] + reg[:, 0] * boxes[:, 2]
    cy = boxes[:, 1] + reg[:, 1] * boxes[:, 3]
    w  = boxes[:, 2] * torch.exp(reg[:, 2])
    h  = boxes[:, 3] * torch.exp(reg[:, 3])

    xmin = cx - w / 2
    ymin = cy - h / 2
    xmax = cx + w / 2
    ymax = cy + h / 2

    decoded = torch.stack([xmin, ymin, xmax, ymax, probs], dim=1)
    return decoded.cpu().detach().numpy()

# ------------------------------------------------------------
# Helper: Draw Boxes on Image
# ------------------------------------------------------------
def draw_boxes(image, gt_boxes, pred_boxes, show_gt=True):
    img_np = np.array(image.copy())
    img_np = cv2.cvtColor(img_np, cv2.COLOR_RGB2BGR)

    # Draw GT boxes in GREEN
    if show_gt and gt_boxes is not None:
        for (x1, y1, x2, y2) in gt_boxes:
            cv2.rectangle(img_np, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)

    # Draw predicted boxes in RED
    for (x1, y1, x2, y2, conf) in pred_boxes:
        cv2.rectangle(img_np, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 2)
        cv2.putText(img_np, f"{conf:.2f}", (int(x1), int(y1)-4),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 255), 1)

    img_rgb = cv2.cvtColor(img_np, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(8,8))
    plt.imshow(img_rgb)
    plt.axis('off')
    plt.show()

# ------------------------------------------------------------
# Test on a Few Validation Images
# ------------------------------------------------------------
num_samples = 5
indices = np.random.choice(len(val_dataset), num_samples, replace=False)

for idx in indices:
    img, target = val_dataset[idx]
    img_in = img.unsqueeze(0).to(device)

    with torch.no_grad():
        cls_pred, reg_pred = model(img_in)

    preds = decode_predictions(cls_pred[0], reg_pred[0], anchors, conf_thresh=0.4)

    # Scale boxes back to image resolution (if needed)
    gt_boxes = target['boxes'].cpu().numpy() if len(target['boxes']) > 0 else None

    print(f"Image {idx} → Detected {len(preds)} faces")
    draw_boxes(transforms.ToPILImage()(img), gt_boxes, preds)
